In [1]:
import torch
import numpy as np
from sklearn.decomposition import PCA, NMF

In [2]:
method = 'SVD'

In [23]:
def dim_fix(tensor, row_len):
            if tensor.shape[0] == row_len:
                print(tensor.shape)

                return tensor
            elif tensor.shape[0] < row_len:
                increased_tensor = torch.zeros((row_len, tensor.shape[1]))
                increased_tensor[: tensor.shape[0], :] = tensor

                if increased_tensor.shape[0] != row_len or increased_tensor.shape[1] != 512:
                    print(increased_tensor.shape)

                return increased_tensor
            else:
                if method == "SVD":
                    U, S, V = torch.svd(tensor)
                    U, S, Vh = torch.linalg.svd(tensor, full_matrices=False)

                    reduced_S = torch.diag(S)[:row_len]
                    reduced_U = U[:, :row_len]
                    reduced_V = Vh[:row_len, :]

                    return torch.mm(
                        torch.mm(
                            reduced_U,
                            reduced_S
                        ),
                        reduced_V
                    )
                elif method == "PCA":
                    vectors_np = torch.detach(tensor).numpy()

                    pca = PCA(n_components=row_len)
                    reduced_tensor = pca.fit_transform(vectors_np)
                    reduced_tensor = torch.from_numpy(reduced_tensor)
                elif method == "NMF":
                    vectors_np = torch.detach(tensor).numpy()

                    nmf = NMF(n_components=row_len)
                    reduced_tensor = nmf.fit_transform(vectors_np)
                    reduced_tensor = torch.from_numpy(reduced_tensor)
                else:
                    raise RuntimeError
            
                if reduced_tensor.shape[0] != row_len or reduced_tensor[1] != 512:
                    print(reduced_tensor.shape)

In [26]:
t1 = torch.randn((10, 6))
dim_fix(t1, 5).shape

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x6 and 5x6)

In [37]:
import torch

# Create a matrix
matrix = torch.Tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Perform SVD
u, s, v = torch.svd(matrix)

# Keep only the first k singular values
k = 2
s_k = s[:k]

# Keep only the first k columns of U and the first k rows of V
u_k = u[:, :k]
v_k = v[:k, :]

tmp = torch.mm(u_k, torch.diag(s_k))
# Reconstruct the original matrix using only the first k singular values and the first k columns of U and V
matrix_reduced = u[:, :k]

print("Original matrix: ", matrix)
print("Reduced matrix: ", matrix_reduced)


Original matrix:  tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])
Reduced matrix:  tensor([[-0.2148,  0.8872],
        [-0.5206,  0.2496],
        [-0.8263, -0.3879]])


In [48]:
t1 = torch.tensor(
    [1., 2., 3.]
).unsqueeze(0)
#t1 = t1.view(1, 1, *t1.shape)

t2 = torch.tensor(
    [
        [1., 2., 3.],
        [4., 5., 6.]
    ]
)
#t2 = t2.view(1, *t2.shape)

t3 = torch.randn(4, 4, 4)

In [49]:
increased_tensor = torch.zeros(4, t2.shape[1])
increased_tensor

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [50]:
t2.shape[1]

3

In [53]:
t2.t()

tensor([[1., 4.],
        [2., 5.],
        [3., 6.]])

In [52]:
increased_tensor[:t2.shape[0], :] = t2
increased_tensor

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [54]:
from sklearn.decomposition import PCA

matrix = np.random.randn(5670, 500)

# Initialize a PCA object with the desired number of components
pca = PCA(n_components=5000)

# Fit the PCA model to the matrix
pca.fit(matrix)

# Transform the matrix and get the first 5000 rows
matrix_transformed = pca.transform(matrix)[:, :5000]

# The shape of the matrix should now be (5000, 786)
print(matrix_transformed.shape)

ValueError: n_components=5000 must be between 0 and min(n_samples, n_features)=500 with svd_solver='full'